<a href="https://colab.research.google.com/github/salaheddinos/Personal_Projects/blob/main/VaccinationsVsCases.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
data = pd.read_excel("owid-covid-data.xlsx")
df = data.copy()

In [ ]:
df.head()

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,new_cases_per_million,new_cases_smoothed_per_million,total_deaths_per_million,new_deaths_per_million,new_deaths_smoothed_per_million,reproduction_rate,icu_patients,icu_patients_per_million,hosp_patients,hosp_patients_per_million,weekly_icu_admissions,weekly_icu_admissions_per_million,weekly_hosp_admissions,weekly_hosp_admissions_per_million,new_tests,total_tests,total_tests_per_thousand,new_tests_per_thousand,new_tests_smoothed,new_tests_smoothed_per_thousand,positive_rate,tests_per_case,tests_units,total_vaccinations,people_vaccinated,people_fully_vaccinated,new_vaccinations,new_vaccinations_smoothed,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,new_vaccinations_smoothed_per_million,stringency_index,population,population_density,median_age,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,excess_mortality
0,AFG,Asia,Afghanistan,2020-02-24,1.0,1.0,NaN,NaN,NaN,NaN,0.026,0.026,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.33,38928341.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN
1,AFG,Asia,Afghanistan,2020-02-25,1.0,0.0,NaN,NaN,NaN,NaN,0.026,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.33,38928341.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN
2,AFG,Asia,Afghanistan,2020-02-26,1.0,0.0,NaN,NaN,NaN,NaN,0.026,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.33,38928341.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN
3,AFG,Asia,Afghanistan,2020-02-27,1.0,0.0,NaN,NaN,NaN,NaN,0.026,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.33,38928341.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN
4,AFG,Asia,Afghanistan,2020-02-28,1.0,0.0,NaN,NaN,NaN,NaN,0.026,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.33,38928341.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN


In [ ]:
#df['date'] = pd.to_datetime(df['date'])

In [ ]:
new_df = df.drop(df[df['continent'].isnull()].index, axis=0)  

In [ ]:
# new_df['new_cases_daily_growth_rate'] = new_df['new_cases'].pct_change(fill_method='ffill') * 100
# new_df['new_deaths_daily_growth_rate'] = new_df['new_deaths'].pct_change(fill_method='ffill') * 100
# pd.set_option('use_inf_as_na', True)

In [ ]:
new_df = new_df[['location', 'date', 'new_cases', 'new_deaths', 'people_fully_vaccinated_per_hundred']] [new_df['date'] > '2021-01-01']

In [ ]:
pd.set_option('display.max_rows', None)
new_df_ = new_df.groupby(['location', new_df['date'].dt.strftime('%Y-%m')])['new_cases', 'new_deaths', 'people_fully_vaccinated_per_hundred'].mean().reset_index()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


In [ ]:
new_df_.rename(columns={"date" : "month", "new_cases" : "avg_monthly_cases", "new_deaths" : "avg_monthly_deaths"}, inplace=True)
# Check shape
new_df_.shape

(1267, 5)

In [ ]:
vaccines_df = pd.read_excel("vaccines_full.xlsx")

In [ ]:
vaccines_df.head()

,location,month,avg_monthly_cases,avg_monthly_deaths,people_fully_vaccinated_per_hundred,% with Pfizer/BioNTech,% with Moderna,% with Oxford/AstraZeneca,% with Johnson&Johnson,% with Sputnik V,% with Sinovac,% with Sinopharm/Beijing,% with CanSino,Vaccines_Used
0,Afghanistan,2020-06,543.241379,16.758621,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sinopharm/Beijing
1,Afghanistan,2020-07,166.387097,17.161290,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sinopharm/Beijing
2,Afghanistan,2020-08,48.193548,3.838710,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sinopharm/Beijing
3,Afghanistan,2020-09,36.966667,1.900000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sinopharm/Beijing
4,Afghanistan,2020-10,69.580645,2.516129,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sinopharm/Beijing


In [ ]:
vaccines = vaccines_df.groupby(['Entity'])[vaccines_df.columns[3:]].max().reset_index()

In [ ]:
vaccines['Total'] = vaccines.sum(axis=1)

In [ ]:
def pct_vacc(df):
  for col in df.columns[1:9]:
    df['% with '+col] = round(df[col] / df['Total'] * 100, 2)
  return df

In [ ]:
vaccines = pct_vacc(vaccines)

In [ ]:
vaccines = vaccines.drop(vaccines.columns[1:10], axis=1)

In [ ]:
vaccines.head()

,Entity,% with Pfizer/BioNTech,% with Moderna,% with Oxford/AstraZeneca,% with Johnson&Johnson,% with Sputnik V,% with Sinovac,% with Sinopharm/Beijing,% with CanSino
0,Austria,71.02,9.93,17.75,1.29,0.0,0.00,0.0,0.0
1,Belgium,69.26,8.57,20.12,2.05,0.0,0.00,0.0,0.0
2,Bulgaria,56.82,12.83,28.96,1.39,0.0,0.00,0.0,0.0
3,Chile,18.06,0.00,1.79,0.00,0.0,79.15,0.0,1.0
4,Croatia,67.41,11.14,20.67,0.78,0.0,0.00,0.0,0.0


In [ ]:
arrays = np.where(vaccines.drop(['Entity'], axis=1)!=0, vaccines.drop(['Entity'], axis=1).columns.values + '-' , None)

In [ ]:
new = []
for array in arrays: 
    new.append(list(filter(None, array)))

In [ ]:
vaccines["Vaccines_Used"] = new

In [ ]:
vaccines["Vaccines_Used"] = [','.join(map(str, l)) for l in vaccines['Vaccines_Used']]

In [ ]:
vaccines['Vaccines_Used'] = vaccines['Vaccines_Used'].str.replace('% with', ' ')

In [ ]:
vaccines_df.rename(columns={"Entity" : "location"}, inplace=True)

In [ ]:
vaccines.head()

,location,% with Pfizer/BioNTech,% with Moderna,% with Oxford/AstraZeneca,% with Johnson&Johnson,% with Sputnik V,% with Sinovac,% with Sinopharm/Beijing,% with CanSino,Vaccines_Used
0,Austria,71.02,9.93,17.75,1.29,0.0,0.00,0.0,0.0,"Pfizer/BioNTech-, Moderna-, Oxford/AstraZe..."
1,Belgium,69.26,8.57,20.12,2.05,0.0,0.00,0.0,0.0,"Pfizer/BioNTech-, Moderna-, Oxford/AstraZe..."
2,Bulgaria,56.82,12.83,28.96,1.39,0.0,0.00,0.0,0.0,"Pfizer/BioNTech-, Moderna-, Oxford/AstraZe..."
3,Chile,18.06,0.00,1.79,0.00,0.0,79.15,0.0,1.0,"Pfizer/BioNTech-, Oxford/AstraZeneca-, Sin..."
4,Croatia,67.41,11.14,20.67,0.78,0.0,0.00,0.0,0.0,"Pfizer/BioNTech-, Moderna-, Oxford/AstraZe..."


In [ ]:
# Join dfs
vaccines_full = pd.merge(new_df_, vaccines_df, on="location", how = "left")

In [ ]:
vaccines_full.head()

,location,month_x,avg_monthly_cases_x,avg_monthly_deaths_x,people_fully_vaccinated_per_hundred_x,month_y,avg_monthly_cases_y,avg_monthly_deaths_y,people_fully_vaccinated_per_hundred_y,% with Pfizer/BioNTech,% with Moderna,% with Oxford/AstraZeneca,% with Johnson&Johnson,% with Sputnik V,% with Sinovac,% with Sinopharm/Beijing,% with CanSino,Vaccines_Used
0,Afghanistan,2021-01,116.566667,6.966667,NaN,2020-06,543.241379,16.758621,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sinopharm/Beijing
1,Afghanistan,2021-01,116.566667,6.966667,NaN,2020-07,166.387097,17.161290,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sinopharm/Beijing
2,Afghanistan,2021-01,116.566667,6.966667,NaN,2020-08,48.193548,3.838710,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sinopharm/Beijing
3,Afghanistan,2021-01,116.566667,6.966667,NaN,2020-09,36.966667,1.900000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sinopharm/Beijing
4,Afghanistan,2021-01,116.566667,6.966667,NaN,2020-10,69.580645,2.516129,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sinopharm/Beijing


In [ ]:
vaccines_full.to_csv("vaccines_2.csv", encoding='utf-8', index=False)

In [ ]:
df = pd.read_excel('vaccines_full.xlsx')

In [ ]:
vaccines_corr = df[(df['people_fully_vaccinated_per_hundred'] > 0) & (df['avg_monthly_cases'] > 0)]

In [ ]:
vaccines_corr.head()

,location,month,avg_monthly_cases,avg_monthly_deaths,people_fully_vaccinated_per_hundred,% with Pfizer/BioNTech,% with Moderna,% with Oxford/AstraZeneca,% with Johnson&Johnson,% with Sputnik V,% with Sinovac,% with Sinopharm/Beijing,% with CanSino,Vaccines_Used
11,Afghanistan,2021-05,390.096774,10.290323,0.246667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sinopharm/Beijing
12,Afghanistan,2021-06,1452.529412,52.823529,0.405000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sinopharm/Beijing
21,Albania,2021-02,1037.142857,14.857143,0.016000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Pfizer/BioNTech, Oxford/AstraZeneca, Sinovac"
24,Albania,2021-05,39.677419,1.838710,8.219333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Pfizer/BioNTech, Oxford/AstraZeneca, Sinovac"
25,Albania,2021-06,9.764706,0.176471,11.247143,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Pfizer/BioNTech, Oxford/AstraZeneca, Sinovac"


In [ ]:
vaccines_grouped = vaccines_corr.groupby('location')[['avg_monthly_cases','people_fully_vaccinated_per_hundred']].corr()

In [ ]:
vaccines_grouped.head()

avg_monthly_cases  people_fully_vaccinated_per_hundred
location                                                                                               
Afghanistan avg_monthly_cases                             1.000000                             1.000000
            people_fully_vaccinated_per_hundred           1.000000                             1.000000
Albania     avg_monthly_cases                             1.000000                            -0.971815
            people_fully_vaccinated_per_hundred          -0.971815                             1.000000
Andorra     avg_monthly_cases                             1.000000                            -0.832433

In [ ]:
vaccines_df = vaccines_grouped.drop('avg_monthly_cases', axis=1).reset_index()

In [ ]:
vaccines_df.head()

,location,level_1,people_fully_vaccinated_per_hundred
0,Afghanistan,avg_monthly_cases,1.000000
1,Afghanistan,people_fully_vaccinated_per_hundred,1.000000
2,Albania,avg_monthly_cases,-0.971815
3,Albania,people_fully_vaccinated_per_hundred,1.000000
4,Andorra,avg_monthly_cases,-0.832433


In [ ]:
vaccines_df = vaccines_df.drop(vaccines_df[vaccines_df['level_1'] == 'people_fully_vaccinated_per_hundred'].index, axis=0)

In [ ]:
vaccines_df.rename(columns = {"people_fully_vaccinated_per_hundred" : "Correlation b/w Average_monthly_cases & Fully_Vaccinated"}, inplace=True)

In [ ]:
vaccines_df.drop("level_1", axis=1, inplace=True)

In [ ]:
vaccines_df.dropna(inplace=True)

In [ ]:
vaccines_df.shape

(143, 2)

In [ ]:
vaccines_df.head()

,location,Correlation b/w Average_monthly_cases & Fully_Vaccinated
0,Afghanistan,1.000000
2,Albania,-0.971815
4,Andorra,-0.832433
6,Angola,-1.000000
8,Antigua and Barbuda,-1.000000


In [ ]:
vaccines_full_2 = pd.merge(vaccines_corr, vaccines_df, on="location", how = "left")

In [ ]:
vaccines_full_2.to_csv("vaccine_full_3.csv", encoding='utf-8', index=False)